In [1]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import SGDRegressor
from explainerdashboard import RegressionExplainer, ClassifierExplainer, ExplainerDashboard
import plotly.express as px
from math import sqrt

RANDOM_STATE = 123

Carga, tratamiento de nulos y combinación de los datasets de SMT y Celonis para el mes de septiembre.

In [11]:
celonis_df = pd.read_excel('data/20230929132544_CELONIS_EXPORT.xlsx')

In [12]:
smt_df = pd.read_excel('data/08_I11_SMT_Incidents_of_own_solution_group(s).xlsx')

smt_df.columns = smt_df.iloc[11].values
smt_df = smt_df.iloc[12:]
smt_df = smt_df.reset_index(drop=True)

c:\ProgramData\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default


In [13]:
smt_df

,Incident ID,Original Incident Number,Requisition ID,Created Date (UTC+0),Incident Type,Status,Status Reason,Company,Customer Department,Region,...,Resolution Categorization Tier 3,Resolution Product Categorization Tier 1,Resolution Product Categorization Tier 2,Resolution Product Categorization Tier 3,Resolution Product Name,Primary Center Code,Target Date,Notes,Resolution,Created by CI-Hotline
0,INC000024939028,NaN,NaN,27.06.2023 12:04,User Service Restoration,Closed,No Further Action Required,GS,GS/HRS2-ES,EMEA,...,- None -,End Device,Personal Computer,Standard Notebook Touch,NaN,933357,02.08.2023 08:00,Q: PROBLEM DESCRIPTION? A:barcode appears wron...,"Se comunica con el usuario, nos indica cerrar ...",Yes
1,INC000025114665,NaN,NaN,17.07.2023 11:27,User Service Request,Closed,No Further Action Required,PS,RBEF/LOG,EMEA,...,- None -,Service,Infrastructure Service,Support Service,NaN,983311,,Su solicitud vía e-mail ha sido transferida a ...,La usuaria nos indica que se ha podido resolve...,Yes
2,INC000025172062,NaN,NaN,25.07.2023 06:59,User Service Request,Closed,No Further Action Required,AA,AA/CTG1-EU,EMEA,...,- None -,End Device,Personal Computer,Standard Notebook Touch,NaN,933411,01.08.2023 09:00,Q: User ID (e.g. xyz8fe): A:LVM5MD Q: Compute...,"Se cambia la placa base, se reinstala el equip...",Yes
3,INC000025184174,NaN,NaN,26.07.2023 09:17,User Service Request,Closed,No Further Action Required,GR,FCM2-Md,EMEA,...,- None -,End Device,Personal Computer,Standard Desktop Tiny,NaN,933481,,Please update software local in FCM departame...,El usuario indica que cerremos la incidencia y...,No
4,INC000025197112,NaN,NaN,27.07.2023 14:20,User Service Request,Closed,No Further Action Required,C,C/CGR-IB2,EMEA,...,- None -,Service,Application Service,Application Service,NaN,NaN,15.08.2023 22:00,Q: User ID (e.g. xyz8fe): A: GOJ5MD Q: Compu...,"Todo el proceso a funcionado correctamente, la...",Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,INC000025416771,NaN,NaN,31.08.2023 15:01,User Service Request,Resolved,No Further Action Required,PS,RBEF/HRL,EMEA,...,- None -,Service,Application Service,Application Service,- None -,983221,,Only for local personal support team Request,"Dear customer, Por favor evalúe el servic...",No
250,INC000025416773,NaN,NaN,31.08.2023 15:02,User Service Restoration,Resolved,No Further Action Required,PS,RBEF/HRL,EMEA,...,- None -,Service,Application Service,Application Service,- None -,983221,,Only for local personal support team Incident,"Dear customer, Por favor evalúe el servic...",No
251,INC000025416876,NaN,NaN,31.08.2023 15:05,User Service Request,Resolved,No Further Action Required,PS,PS/MFT-PLA,EMEA,...,- None -,Service,Business Service,Business Service,- None -,NaN,,Only for local personal support team Request,"Dear customer, You receive this mail as autom...",Yes
252,INC000025416879,NaN,NaN,31.08.2023 15:06,User Service Request,Resolved,No Further Action Required,PS,PS/MFT-PLA,EMEA,...,- None -,Service,Application Service,Application Service,- None -,NaN,,Only for local personal support team Request,"Dear customer, You receive this mail as autom...",Yes


In [14]:
celonis_df

,Incident Number,Submit Date,Service Name,Site Group,Site,Incident Type,Ticket Summary,Submitter Group,Last Assigned Organization,Last Assigned Group,...,Last Resolution Date,Template Name,User Company,Resolver Organization,Requisition ID,Reported Source,Last Resolution Time,CI,Status,Priority
0,INC000025219797,2023-08-01 06:37:22,MOZILLA FIREFOX |EMEA,ES,Md,User Service Request,How to clear cache in Mozilla Firefox manually,ITSD - IT Service Desk,Service Solutions,ITSD - ES,...,2023-08-01 06:37:22,NaN,EU1,CI-Hotline / ITSD 1st Level,NaN,Phone,0.280556,NaN,Closed,Low
1,INC000025219902,2023-08-01 06:48:05,SCCM PACKAGE INSTALLATION FAILURE,ES,Md,User Service Restoration,LEN_Hotkey_9.2.0.5_Tx90_W10 - RBHW9390,ITSD - IT Service Desk,Service Solutions,ITSD - SCCM Installation Support,...,2023-08-21 06:42:56,NaN,C,CI-Hotline / ITSD 2nd Level,NaN,Phone,480.225833,SCCM CLIENT HARDWARE DRIVERS,Closed,Low
2,INC000025221003,2023-08-01 07:36:48,BUSINESS-OBJECTS_APPLICATION,ES,Md1,User Service Restoration,Excel - Analysis Error,ITSD - IT Service Desk,Data and Digital,BI BO Platform support,...,2023-08-01 09:18:30,NaN,AE,Rest Support Organizations,NaN,Phone,1.920278,BUSINESS-OBJECTS_APPLICATION,Closed,Low
3,INC000025221039,2023-08-01 07:48:04,ITSD DOCUMENTED,ES,Md1,User Service Request,Autocad License problem,ITSD - IT Service Desk,Service Solutions,ITSD - ES,...,2023-09-01 12:53:00,NaN,BMG,CI-Hotline / ITSD 1st Level,NaN,Chat,749.219167,NaN,Closed,Medium
4,INC000025221092,2023-08-01 07:59:17,MY COMPUTER - LOCAL SUPPORT |WORLD,ES,Md1,User Service Request,Equipo MD-C-00029 sin espacio libre,ITSD - IT Service Desk,Regional Services EMEA,Spain1 - EXT,...,2023-08-01 12:34:45,NaN,AE,IT Personal Support,NaN,Email,4.697778,MD-C-00029,Closed,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,INC000025416134,2023-08-31 13:41:35,WCMS BGN,ES,Md1,User Service Request,[C7] The user cannot enter Sugerencias,ITSD - IT Service Desk,RBEI,CENTRAL inside Portal Advisor RBEI - EXT,...,2023-09-06 05:56:07,NaN,AE,Rest Support Organizations,NaN,Phone,136.461667,NaN,Closed,Low
436,INC000025416227,2023-08-31 13:31:13,IDM - USERADMINISTRATION | ITSP ORDERFORM ACTI...,ES,Md1,User Service Request,Manage access rights via IdM,IT Infrastructure Automation,Service Solutions,ITSD - IDM Expert Team,...,2023-08-31 16:22:53,S2T_Autobus_SNOW_Request,AE,CI-Hotline / ITSD 2nd Level,RITM1414984,Self Service,2.861111,NaN,Closed,Low
437,INC000025416602,2023-08-31 14:20:30,IDENTITIES (USERS) NEW / CHANGE / DELETE,ES,Md,User Service Request,User Prolongation Order RITM1412095 required ...,IT Infrastructure Automation,Digital Workplace,Identity Lifecycle Support,...,2023-09-08 10:35:36,S2T_Autobus_Request,BD,Rest Support Organizations,NaN,Systems Management,188.251667,NaN,Resolved,Low
438,INC000025417081,2023-08-31 16:47:12,SCANSYSTEM,ES,Md1,User Service Restoration,No abre programa Visual Factory Scansistem,ITSD - IT Service Desk,Regional Services EMEA,Spain1 - EXT,...,2023-09-01 11:40:07,NaN,AE,IT Personal Support,NaN,Email,18.953333,MD-C-000FP,Closed,Low


In [15]:
celonis_df = celonis_df.drop(['Template Name', 'Requisition ID', 'CI'], axis=1)
celonis_df.dropna(inplace=True)
celonis_df = celonis_df.drop(['Submit Date', 'Last Resolution Date', 'Ticket Summary'], axis=1)

In [16]:
celonis_df

,Incident Number,Service Name,Site Group,Site,Incident Type,Submitter Group,Last Assigned Organization,Last Assigned Group,Entry Channel,User Company,Resolver Organization,Reported Source,Last Resolution Time,Status,Priority
0,INC000025219797,MOZILLA FIREFOX |EMEA,ES,Md,User Service Request,ITSD - IT Service Desk,Service Solutions,ITSD - ES,CI-Hotline / ITSD,EU1,CI-Hotline / ITSD 1st Level,Phone,0.280556,Closed,Low
1,INC000025219902,SCCM PACKAGE INSTALLATION FAILURE,ES,Md,User Service Restoration,ITSD - IT Service Desk,Service Solutions,ITSD - SCCM Installation Support,CI-Hotline / ITSD,C,CI-Hotline / ITSD 2nd Level,Phone,480.225833,Closed,Low
2,INC000025221003,BUSINESS-OBJECTS_APPLICATION,ES,Md1,User Service Restoration,ITSD - IT Service Desk,Data and Digital,BI BO Platform support,CI-Hotline / ITSD,AE,Rest Support Organizations,Phone,1.920278,Closed,Low
3,INC000025221039,ITSD DOCUMENTED,ES,Md1,User Service Request,ITSD - IT Service Desk,Service Solutions,ITSD - ES,CI-Hotline / ITSD,BMG,CI-Hotline / ITSD 1st Level,Chat,749.219167,Closed,Medium
4,INC000025221092,MY COMPUTER - LOCAL SUPPORT |WORLD,ES,Md1,User Service Request,ITSD - IT Service Desk,Regional Services EMEA,Spain1 - EXT,CI-Hotline / ITSD,AE,IT Personal Support,Email,4.697778,Closed,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,INC000025416134,WCMS BGN,ES,Md1,User Service Request,ITSD - IT Service Desk,RBEI,CENTRAL inside Portal Advisor RBEI - EXT,CI-Hotline / ITSD,AE,Rest Support Organizations,Phone,136.461667,Closed,Low
436,INC000025416227,IDM - USERADMINISTRATION | ITSP ORDERFORM ACTI...,ES,Md1,User Service Request,IT Infrastructure Automation,Service Solutions,ITSD - IDM Expert Team,IT Service Portal,AE,CI-Hotline / ITSD 2nd Level,Self Service,2.861111,Closed,Low
437,INC000025416602,IDENTITIES (USERS) NEW / CHANGE / DELETE,ES,Md,User Service Request,IT Infrastructure Automation,Digital Workplace,Identity Lifecycle Support,AutomationBus,BD,Rest Support Organizations,Systems Management,188.251667,Resolved,Low
438,INC000025417081,SCANSYSTEM,ES,Md1,User Service Restoration,ITSD - IT Service Desk,Regional Services EMEA,Spain1 - EXT,CI-Hotline / ITSD,AE,IT Personal Support,Email,18.953333,Closed,Low


In [17]:
smt_df = smt_df.drop(['Original Incident Number', 'Requisition ID', 'CI+', 'Product Name', 
                      'Resolution Product Name', 'Primary Center Code'], axis=1)
smt_df.dropna(inplace=True)
smt_df = smt_df.drop(['Created Date (UTC+0)', 'Summary', 'Last Modified Date (UTC+0)', 
                      'First Resolved Date (UTC+0)', 'Last Resolved Date (UTC+0)', 'Notes', 'Resolution'], axis=1)

Eliminamos las columnas que aparecen repetidas en ambos datasets.

In [18]:
smt_df = smt_df.drop(['Site Group', 'Site', 'Incident Type', 'Reported Source', 'Status'], axis=1)

Observamos que existen columnas, como Customer Department, con gran cantidad de posibles valores.

In [19]:
ocurrencias = smt_df['Customer Department'].value_counts()
print(len(ocurrencias))
print(ocurrencias)

117
BD/PFE4-IA     10
RBEM/LOM-Q      9
RBEF/MFE12      9
RBEF/TEF1       8
RBEF/CTG        7
               ..
HC/SEI-MK       1
C/CMR-IB        1
AA/CTG1-EU      1
RBEM/TEF8.1     1
AA/SWS2-EI      1
Name: Customer Department, Length: 117, dtype: int64


In [20]:
combined_df = pd.merge(celonis_df, smt_df, how='inner', left_on='Incident Number', right_on='Incident ID')

Para reducir el número de columnas tras utilizar onehot, eliminamos las columnas con muchos posibles valores y las columnas de tiempo. Las columnas que aparecen en ambos datasets han sido eliminadas previamente del dataframe de SMT.

In [21]:
combined_df.drop(['Service Name', 'Incident ID','In Progess Time (hhh:mi)\n', 'Actual Duration/Open Time (hhh:mi)', 'Customer Department', 'Desk Location', 
                  'Down Time of CI-Unavailability (hhh:mi)', 'Service+', 'Target Date'], axis=1, inplace=True)

In [22]:
pd.set_option('display.max_columns', None)
combined_df

,Incident Number,Site Group,Site,Incident Type,Submitter Group,Last Assigned Organization,Last Assigned Group,Entry Channel,User Company,Resolver Organization,Reported Source,Last Resolution Time,Status,Priority,Status Reason,Company,Region,Impact,Open Priority,Current Priority,Assigned Group,Assigned Group Department,Operational Categorization Tier 1,Operational Categorization Tier 2,Operational Categorization Tier 3,Product Categorization Tier 1,Product Categorization Tier 2,Product Categorization Tier 3,Resolution Categorization Tier 1,Resolution Categorization Tier 2,Resolution Categorization Tier 3,Resolution Product Categorization Tier 1,Resolution Product Categorization Tier 2,Resolution Product Categorization Tier 3,Created by CI-Hotline
0,INC000025221092,ES,Md1,User Service Request,ITSD - IT Service Desk,Regional Services EMEA,Spain1 - EXT,CI-Hotline / ITSD,AE,IT Personal Support,Email,4.697778,Closed,Low,No Further Action Required,AE,EMEA,4-Minor/Localized,Low,Low,Spain1 - EXT,BD/PFE-IA,Request,- None -,- None -,Service,End Device Service,Personal Computer,Application,Performance,- None -,End Device,Personal Computer,Standard Desktop Tiny,Yes
1,INC000025221966,ES,Md,User Service Request,ITSD - IT Service Desk,Regional Services EMEA,Spain1 - EXT,CI-Hotline / ITSD,EU2,IT Personal Support,Phone,4.116667,Closed,Low,No Further Action Required,EU2,EMEA,4-Minor/Localized,Low,Low,Spain1 - EXT,BD/PFE-IA,Request,- None -,- None -,Service,End Device Service,Personal Computer,Application,Performance,- None -,End Device,Personal Computer,Standard Notebook Touch,Yes
2,INC000025223159,ES,Md1,User Service Request,ITSD - IT Service Desk,Regional Services EMEA,Spain1 - EXT,CI-Hotline / ITSD,AE,IT Personal Support,Email,23.067500,Closed,Low,No Further Action Required,AE,EMEA,4-Minor/Localized,Low,Low,Spain1 - EXT,BD/PFE-IA,Request,- None -,- None -,Service,Business Service,Business Service,Infrastructure,Hardware,- None -,End Device,Personal Computer,Standard Desktop Tiny,Yes
3,INC000025223804,ES,Md1,User Service Request,ITSD - IT Service Desk,Regional Services EMEA,Spain1 - EXT,CI-Hotline / ITSD,AE,IT Personal Support,Email,22.799722,Closed,Low,No Further Action Required,AE,EMEA,4-Minor/Localized,Low,Low,Spain1 - EXT,BD/PFE-IA,Request,- None -,- None -,Service,IT Service,IT Service,Application,Performance,- None -,End Device,Personal Computer,Standard Desktop Tiny,Yes
4,INC000025224626,ES,Md,User Service Request,ITSD - IT Service Desk,Regional Services EMEA,Spain1,IT Service Portal,AA,IT Personal Support,Self Service,24.208611,Closed,Low,No Further Action Required,AA,EMEA,4-Minor/Localized,Low,Low,Spain1,BD/PFE-IA,Request,- None -,- None -,Service,End Device Service,Mobile Device Service,Request,Hardware,- None -,Service,End Device Service,Mobile Device Service,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,INC000025414574,ES,Md1,User Service Restoration,Spain1,Regional Services EMEA,Spain1,IT Personal Support,AE,IT Personal Support,Walk In,0.017778,Closed,Low,No Further Action Required,AE,EMEA,4-Minor/Localized,Low,Low,Spain1,BD/PFE-IA,Failure,- None -,- None -,Service,IT Service,IT Service,Application,- None -,- None -,Service,IT Service,IT Service,No
164,INC000025415111,ES,Md1,User Service Restoration,Spain1,Regional Services EMEA,Spain1,IT Personal Support,AE,IT Personal Support,Walk In,0.003333,Closed,Low,No Further Action Required,AE,EMEA,4-Minor/Localized,Low,Low,Spain1,BD/PFE-IA,Failure,- None -,- None -,Service,End Device Service,Personal Computer,Application,- None -,- None -,End Device,Personal Computer,Convertible Notebook,Yes
165,INC000025415115,ES,Md,User Service Request,Spain1,Regional Services EMEA,Spain1,IT Personal Support,GR,IT Personal Support,Walk In,0.001389,Closed,Low,No Further Action Required,GR,EMEA,4-Minor/Localized,Low,Low,Spain1,BD/PFE-IA,Request,- None -,- None -,Service,End Device Service,Mobile Devic

Aplicamos onehot.

In [23]:
columns_list = combined_df.columns
columns_list = [elem for elem in columns_list if elem not in ['Incident Number', 'Last Resolution Time', 'Incident ID']] 
combined_df_one_hot = pd.get_dummies(combined_df, columns=columns_list)
combined_df_one_hot

,Incident Number,Last Resolution Time,Site Group_ES,Site_Md,Site_Md1,Incident Type_Infrastructure Restoration,Incident Type_User Service Request,Incident Type_User Service Restoration,Submitter Group_Human Resources - Personnel Administration,Submitter Group_IT Infrastructure Automation,Submitter Group_ITSD - IT Service Desk,Submitter Group_Spain1,Submitter Group_Spain1 - EXT,Last Assigned Organization_Regional Services EMEA,Last Assigned Group_Spain1,Last Assigned Group_Spain1 - EXT,Entry Channel_AutomationBus,Entry Channel_CI-Hotline / ITSD,Entry Channel_IT Personal Support,Entry Channel_IT Service Portal,Entry Channel_IT Support Page,Entry Channel_Other,User Company_AA,User Company_AE,User Company_BD,User Company_BMG,User Company_BT,User Company_C,User Company_EU1,User Company_EU2,User Company_GR,User Company_GS,User Company_HC,User Company_M,User Company_PT,User Company_SO,Resolver Organization_IT Personal Support,Reported Source_Chat,Reported Source_Direct Input,Reported Source_Email,Reported Source_Phone,Reported Source_Self Service,Reported Source_Systems Management,Reported Source_Walk In,Status_Closed,Priority_Low,Priority_Medium,Status Reason_Customer Follow-Up Required,Status Reason_No Further Action Required,Company_AA,Company_AE,Company_BD,Company_BMG,Company_BT,Company_C,Company_EU1,Company_EU2,Company_GR,Company_GS,Company_HC,Company_M,Company_PT,Company_SO,Region_EMEA,Impact_3-Moderate/Limited,Impact_4-Minor/Localized,Open Priority_-1,Open Priority_Low,Open Priority_Medium,Current Priority_Low,Current Priority_Medium,Assigned Group_Spain1,Assigned Group_Spain1 - EXT,Assigned Group Department_BD/PFE-IA,Operational Categorization Tier 1_Failure,Operational Categorization Tier 1_Request,Operational Categorization Tier 2_- None -,Operational Categorization Tier 2_Add,Operational Categorization Tier 2_Application,Operational Categorization Tier 2_Consult,Operational Categorization Tier 2_Reset,Operational Categorization Tier 3_- None -,Operational Categorization Tier 3_Password,Product Categorization Tier 1_Service,Product Categorization Tier 2_Application Service,Product Categorization Tier 2_Business Service,Product Categorization Tier 2_Commercial Service,Product Categorization Tier 2_End Device Service,Product Categorization Tier 2_IT Service,Product Categorization Tier 2_Infrastructure Service,Product Categorization Tier 3_Application Service,Product Categorization Tier 3_Business Service,Product Categorization Tier 3_Communication Service,Product Categorization Tier 3_File Service,Product Categorization Tier 3_IT Service,Product Categorization Tier 3_Local Software Service,Product Categorization Tier 3_Mobile Device Service,Product Categorization Tier 3_Peacy Package,Product Categorization Tier 3_Personal Computer,Product Categorization Tier 3_Print Service,Product Categorization Tier 3_SAP Business Service,Product Categorization Tier 3_Security Service,Product Categorization Tier 3_Support Service,Resolution Categorization Tier 1_Application,Resolution Categorization Tier 1_Handling,Resolution Categorization Tier 1_Infrastructure,Resolution Categorization Tier 1_Request,Resolution Categorization Tier 2_- None -,Resolution Categorization Tier 2_Documentation,Resolution Categorization Tier 2_Education,Resolution Categorization Tier 2_Hardware,Resolution Categorization Tier 2_Network Performance,Resolution Categorization Tier 2_Performance,Resolution Categorization Tier 2_Software,Resolution Categorization Tier 2_Unclear,Resolution Categorization Tier 2_User Error,Resolution Categorization Tier 2_User Rights,Resolution Categorization Tier 3_- None -,Resolution Product Categorization Tier 1_End Device,Resolution Product Categorization Tier 1_Service,Resolution Product Categorization Tier 2_Application Service,Resolution Product Categorization Tier 2_Business Service,Resolution Product Categorization Tier 2_Commercial Service,Resolution Product Categorization Tier 2_End Device Service,Resolution Product Categoriza

Se aprecia que el número de columnas ha disminuido drásticamente.

Dividimos en conjunto de entrenamiento y prueba.

In [24]:
X = combined_df_one_hot.drop(['Incident Number', 'Last Resolution Time'], axis=1).values
y = combined_df_one_hot['Last Resolution Time'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

Creamos, entrenamos y evaluamos el modelo de Gradient Boosting.

In [25]:
gbr = GradientBoostingRegressor(random_state=RANDOM_STATE)

gbr.fit(X_train, y_train)

predictions = gbr.predict(X_test)

mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
print("Error cuadrático medio (MSE) en el conjunto de prueba: {:.4f}".format(mse))
print("Raíz del error cuadrático medio (RMSE) en el conjunto de prueba: {:.4f}".format(sqrt(mse)))
print("Error medio absoluto (MAE) en el conjunto de prueba: {:.4f}".format(mae))

predictions

Error cuadrático medio (MSE) en el conjunto de prueba: 849.8895
Raíz del error cuadrático medio (RMSE) en el conjunto de prueba: 29.1529
Error medio absoluto (MAE) en el conjunto de prueba: 13.4342


array([-2.72513033e-01,  2.91524947e+01, -2.72513033e-01,  2.40445449e+01,
        3.14676113e+01, -3.07500128e-01,  3.02382117e+01, -4.00361443e-02,
        9.11554561e+00, -3.07500128e-01, -5.04904990e-03,  5.97488589e+00,
        3.09044137e+01,  6.00102478e+00, -2.72513033e-01,  5.97488589e+00,
        2.85367979e+01, -4.00361443e-02,  3.43122522e+01, -1.12575317e+00,
       -3.07500128e-01,  4.91336508e+00, -2.72513033e-01,  5.97488589e+00,
        1.99313172e+01,  8.92637235e+01,  2.42943341e+01, -1.09076608e+00,
        4.67427006e+01, -2.72513033e-01,  1.47172993e+02,  4.99293082e+01,
       -4.00361443e-02,  5.97488589e+00])

A continuación repetimos el proceso con el algoritmo de descenso del gradiente estocástico.

In [26]:
sgd = SGDRegressor(random_state=RANDOM_STATE)

sgd.fit(X_train, y_train)

predictions = sgd.predict(X_test)

mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
print("Error cuadrático medio (MSE) en el conjunto de prueba: {:.4f}".format(mse))
print("Raíz del error cuadrático medio (RMSE) en el conjunto de prueba: {:.4f}".format(sqrt(mse)))
print("Error medio absoluto (MAE) en el conjunto de prueba: {:.4f}".format(mae))

predictions

Error cuadrático medio (MSE) en el conjunto de prueba: 701.0608
Raíz del error cuadrático medio (RMSE) en el conjunto de prueba: 26.4776
Error medio absoluto (MAE) en el conjunto de prueba: 17.9272


array([ -1.19002219,  75.91767526,  -1.72726936,  58.23385656,
        49.98837326,  -5.33901156,  52.32305508,  -0.50151735,
        -8.94945979,  -0.12964823,   0.63113601,   6.8579863 ,
        31.41299426,   7.31188245,  28.66912809,   6.8579863 ,
        53.57893771,  -4.39746853,  68.80668571,  19.19715165,
         1.20826142, -10.20402845,  29.16668769,   3.65640055,
        -6.68466492,  53.96078448,  30.57418773,  16.81278722,
        85.33120378,  -1.72726936,  92.59011831,  74.10337547,
        -4.39746853,  19.09916384])

Creamos los dataframes.

In [22]:
X_test_df = pd.DataFrame(X_test, columns=combined_df_one_hot.drop(['Incident Number', 'Last Resolution Time'], axis=1).columns)
y_test_df = pd.DataFrame(y_test, columns=['Last Resolution Time'])

Generamos ambos explainerdashboard para poder interpretar los modelos

In [ ]:
explainer = RegressionExplainer(gbr, X_test_df, y_test_df)

ExplainerDashboard(explainer).run(port=8051)


In [ ]:
explainer = RegressionExplainer(sgd, X_test_df, y_test_df)

ExplainerDashboard(explainer).run(port=8052)